In [1]:
import os
import requests
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

In [2]:
from dotenv import load_dotenv
load_dotenv()
from crewai import LLM

model_provider = os.getenv("MODEL_PROVIDER", "openai").lower()

if model_provider == "ollama":
    llm = LLM(
        model=f"ollama/{os.getenv('OLLAMA_MODEL', 'llama2')}",
        base_url=os.getenv("OLLAMA_API_URL", "http://localhost:11434/api"),
    )
else:
    llm = LLM(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        api_key=os.getenv("OPENAI_API_KEY"),
    )
    

In [3]:
'''
from crewai import Agent
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama2")
'''

'\nfrom crewai import Agent\nfrom langchain_ollama import OllamaLLM\n\nllm = OllamaLLM(model="llama2")\n'

In [4]:
print(f"Using {model_provider.upper()} via {llm.model}")

Using OLLAMA via ollama/llama2


In [5]:
print(llm.call("Say hello from the currency analyst agent!"))

Hello there! *adjusts glasses* As a currency analyst agent, I'm here to provide you with the latest updates on currencies and exchange rates. How can I assist you today?


In [6]:
class CurrencyConverterInput(BaseModel):
    """Input schema for CurrencyConverterTool."""
    amount: float = Field(..., description="The amount to convert.")
    from_currency: str = Field(..., description="The source currency code (e.g., 'USD').")
    to_currency: str = Field(..., description="The target currency code (e.g., 'EUR').")


In [7]:
class CurrencyConverterTool(BaseTool):
    name: str = "currency_converter"
    description: str = "Converts an amount from one currency to another."
    args_schema: Type[BaseModel] = CurrencyConverterInput

    def _run(self, amount: float, from_currency: str, to_currency: str) -> str:
        url = f"https://api.exchangerate.host/convert?from={from_currency}&to={to_currency}&amount={amount}"
        response = requests.get(url)
        if response.status_code != 200:
            return "Failed to fetch exchange rates."
        data = response.json()
        if "result" not in data:
            return "Invalid currency codes or API error."
        converted_amount = data["result"]
        return f"{amount} {from_currency} = {converted_amount:.2f} {to_currency}"


In [8]:
from crewai import Agent

currency_analyst = Agent(
    role="Currency Analyst",
    goal="Provide real-time currency conversions and financial insights.",
    backstory=(
        "You are a finance expert with deep knowledge of global exchange rates."
        "You help users with currency conversion and financial decision-making."
    ),
    tools=[CurrencyConverterTool()],
    allow_delegation=False,
    llm = llm,
    verbose=True
)


In [9]:
from crewai import Task

currency_conversion_task = Task(
    description=(
        "Call the 'currency_converter' tool directly with the parameters: "
        "amount={amount}, from_currency={from_currency}, to_currency={to_currency}. "
        "Then explain the exchange rate and a short financial insight."
    ),
    expected_output=(
        "A detailed response including the converted amount, "
        "exchange rate used, and short financial insight."
    ),
    agent=currency_analyst
)



In [10]:
from crewai import Crew, Process

crew = Crew(
    agents=[currency_analyst],
    tasks=[currency_conversion_task],
    process=Process.sequential
)

response = crew.kickoff(inputs={
    "amount": 100,
    "from_currency": "USD",
    "to_currency": "COP"
})

print(response)



Action 'Call the 'currency_converter' tool with the parameters 'amount=100', 'from_currency=USD', and 'to_currency=COP'.' don't exist, these are the only available Actions:
Tool Name: currency_converter
Tool Arguments: {'amount': {'description': 'The amount to convert.', 'type': 'float'}, 'from_currency': {'description': "The source currency code (e.g., 'USD').", 'type': 'str'}, 'to_currency': {'description': "The target currency code (e.g., 'EUR').", 'type': 'str'}}
Tool Description: Converts an amount from one currency to another.



Action 'Call the 'currency_converter' tool with the parameters 'amount=100', 'from_currency=USD', and 'to_currency=COP'.' don't exist, these are the only available Actions:
Tool Name: currency_converter
Tool Arguments: {'amount': {'description': 'The amount to convert.', 'type': 'float'}, 'from_currency': {'description': "The source currency code (e.g., 'USD').", 'type': 'str'}, 'to_currency': {'description': "The target currency code (e.g., 'EUR').", 

In [ ]:
response = currency_analyst.llm.call("Dime una breve descripción del dolar frente al peso colombiano.")
print(response)